# 84 - Monopoly Odds

## Problem Statement

<p>In the game, <strong>Monopoly</strong>, the standard board is set up in the following way:</p>
<div class="center">
<img src="0084_monopoly_board.png?1678992052" alt="0084_monopoly_board.png">
</div>
<p>A player starts on the GO square and adds the scores on two 6-sided dice to determine the number of squares they advance in a clockwise direction. Without any further rules we would expect to visit each square with equal probability: 2.5%. However, landing on G2J (Go To Jail), CC (community chest), and CH (chance) changes this distribution.</p>
<p>In addition to G2J, and one card from each of CC and CH, that orders the player to go directly to jail, if a player rolls three consecutive doubles, they do not advance the result of their 3rd roll. Instead they proceed directly to jail.</p>
<p>At the beginning of the game, the CC and CH cards are shuffled. When a player lands on CC or CH they take a card from the top of the respective pile and, after following the instructions, it is returned to the bottom of the pile. There are sixteen cards in each pile, but for the purpose of this problem we are only concerned with cards that order a movement; any instruction not concerned with movement will be ignored and the player will remain on the CC/CH square.</p>
<ul><li>Community Chest (2/16 cards):
<ol><li>Advance to GO</li>
<li>Go to JAIL</li>
</ol></li>
<li>Chance (10/16 cards):
<ol><li>Advance to GO</li>
<li>Go to JAIL</li>
<li>Go to C1</li>
<li>Go to E3</li>
<li>Go to H2</li>
<li>Go to R1</li>
<li>Go to next R (railway company)</li>
<li>Go to next R</li>
<li>Go to next U (utility company)</li>
<li>Go back 3 squares.</li>
</ol></li>
</ul><p>The heart of this problem concerns the likelihood of visiting a particular square. That is, the probability of finishing at that square after a roll. For this reason it should be clear that, with the exception of G2J for which the probability of finishing on it is zero, the CH squares will have the lowest probabilities, as 5/8 request a movement to another square, and it is the final square that the player finishes at on each roll that we are interested in. We shall make no distinction between "Just Visiting" and being sent to JAIL, and we shall also ignore the rule about requiring a double to "get out of jail", assuming that they pay to get out on their next turn.</p>
<p>By starting at GO and numbering the squares sequentially from 00 to 39 we can concatenate these two-digit numbers to produce strings that correspond with sets of squares.</p>
<p>Statistically it can be shown that the three most popular squares, in order, are JAIL (6.24%) = Square 10, E3 (3.18%) = Square 24, and GO (3.09%) = Square 00. So these three most popular squares can be listed with the six-digit modal string: 102400.</p>
<p>If, instead of using two 6-sided dice, two 4-sided dice are used, find the six-digit modal string.</p>

## Solution

The answer is found with Monte Carlo simulation. 

In [14]:
import random

def roll_dice(sides=4):
    return random.randint(1, sides), random.randint(1, sides)

def move_to_next_r(current_pos):
    if current_pos < 5 or current_pos >= 35: return 5
    elif current_pos < 15: return 15
    elif current_pos < 25: return 25
    elif current_pos < 35: return 35

def move_to_next_u(current_pos):
    if current_pos < 12 or current_pos >= 28: return 12
    else: return 28

def simulate_monopoly(turns=1000000, sides=4):
    board_size = 40
    frequency = [0] * board_size
    position = 0
    consecutive_doubles = 0

    cc_cards = ['GO', 'JAIL'] + ['NONE'] * 14
    ch_cards = ['GO', 'JAIL', 'C1', 'E3', 'H2', 'R1', 'NEXT_R', 'NEXT_R', 'NEXT_U', 'BACK_3'] + ['NONE'] * 6

    random.shuffle(cc_cards)
    random.shuffle(ch_cards)

    cc_index = 0
    ch_index = 0

    for _ in range(turns):
        dice1, dice2 = roll_dice(sides)
        if dice1 == dice2:
            consecutive_doubles += 1
        else:
            consecutive_doubles = 0

        if consecutive_doubles >= 3:
            position = 10  # Go to jail
            consecutive_doubles = 0
        else:
            position = (position + dice1 + dice2) % board_size

        if position == 30:  # G2J
            position = 10  # Jail

        if position in [2, 17, 33]:  # Community Chest
            card = cc_cards[cc_index]
            if card == 'GO':
                position = 0
            elif card == 'JAIL':
                position = 10
            cc_index = (cc_index + 1) % len(cc_cards)

        if position in [7, 22, 36]:  # Chance
            card = ch_cards[ch_index]
            if card == 'GO':
                position = 0
            elif card == 'JAIL':
                position = 10
            elif card == 'C1':
                position = 11
            elif card == 'E3':
                position = 24
            elif card == 'H2':
                position = 39
            elif card == 'R1':
                position = 5
            elif card == 'NEXT_R':
                position = move_to_next_r(position)
            elif card == 'NEXT_U':
                position = move_to_next_u(position)
            elif card == 'BACK_3':
                position = (position - 3) % board_size
            ch_index = (ch_index + 1) % len(ch_cards)

        frequency[position] += 1

    return frequency

# Run the simulation
frequency = simulate_monopoly(turns=10**6)
top_three = sorted(range(len(frequency)), key=lambda x: frequency[x], reverse=True)[:3]
modal_string = ''.join(f"{x:02d}" for x in top_three)
modal_string

'101524'